# SV Hackathon 
### October 12, 2022
### Current update on Oct. 12 2022
### NIST-GREX" Genome Regions Exploration


### Starting with S1: (Updating GRCh38 resources to T2T-CHM13v2.0 reference) Generate T2T-CHM13v2.0 stratification BED file similar to what exists for GRCh38
- Running GEM on CHRM13v.2.0
- Testing it out first  on chr21, then maybe moving on to whole genome


### Setting up our environment and tools

In [ ]:
export MAIN=/path/to/
# This was all done on NIH's Biowulf Cluster

Upload CHM13v2.0 fasta from locally downloaded source

In [ ]:
# chm13v2.0.fa.gz
# chm13v2.0.fa.gz.gzi


Extract chr 21 for testing

In [ ]:
module load samtools
samtools faidx chm13v2.0.fa.gz chr21 > chm13_chr21.fa.gz

In [ ]:
samtools faidx chm13_chr21.fa.gz

Make Hackathon directory for all fasta and output files

In [ ]:
mkdir $MAIN/sv_hackathon/
mv chm13* $MAIN/sv_hackathon/

In [ ]:
tar jxvf GEM-binaries-Linux-x86_64-core_i3-20130406-045632.tbz2

Upload and install bedops binaries

In [ ]:
mkdir $MAIN/bedops
cp bin/* $MAIN/bedops/
rm -r $MAIN/bin/

Upload and install gem binaries

In [ ]:
tar jxvf GEM-binaries-Linux-x86_64-core_i3-20130406-045632.tbz2

All binaries and files are now in directory

### Time to import GEM .sh script 
https://github.com/genome-in-a-bottle/genome-stratifications/blob/master/GRCh38/mappability/run_GEM_mappability_GRCh38.sh

In [ ]:
wget https://github.com/genome-in-a-bottle/genome-stratifications/blob/master/GRCh38/mappability/run_GEM_mappability_GRCh38.sh
mv run_GEM_mappability_GRCh38.sh gem_mappability.sh

Run for chr21
Run twice each time
-l 100 -m 2 -e 1 \
-l 250 -m 0 -e 0 

In [ ]:
sbatch gem_mappability.sh 
# Altered paths and -l -m -e directly in script

In [ ]:
Output to move forward chm13_chr21_gemmap_l100_m2_e1_uniq.bed and chm13_chr21_gemmap_l250_m0_e0_uniq.bed

## Need to next run the second script for union files for validation

https://github.com/genome-in-a-bottle/genome-stratifications/blob/master/GRCh38/mappability/run_GEM_mappability_sort_GRCh38.sh

In [ ]:
wget https://github.com/genome-in-a-bottle/genome-stratifications/blob/master/GRCh38/mappability/run_GEM_mappability_sort_GRCh38.sh
mv run_GEM_mappability_sort_GRCh38.sh sort_union.sh

In [ ]:
# Create bed file from reference
awk 'BEGIN {FS="\t"}; {print $1 FS "0" FS $2}' $MAIN/sv_hackathon/chm13_chr21.fa.gz.fai > $MAIN/sv_hackathon/chm13_chr21.bed
head $MAIN/sv_hackathon/chm13_chr21.bed

In [ ]:
# Create genome file from reference
awk '{FS="\t"};{print $1 FS $3}' $MAIN/sv_hackathon/chm13_chr21.bed > $MAIN/sv_hackathon/chm13_chr21_onlychr.genome
head $MAIN/sv_hackathon/chm13_chr21_onlychr.genome

In [ ]:
sbatch --mem=100g --cpus-per-task=4 --mail-type=ALL --time=12:00:00 sort_union.sh chm13_chr21_gemmap_l100_m2_e1_uniq.bed chm13_chr21_gemmap_l250_m0_e0_uniq.bed

## Run for whole genome

In [ ]:
# Remember to alter path names and stringency levels
# Run L1-L49 (block out the rest)
sbatch gem_mappability.sh

In [ ]:
# Checking where each chromosome starts
	grep -n "chr1" chm13_gemmap_l100_m2_e1_name_clean.wig | head -n 2
	grep -n "chr2" chm13_gemmap_l100_m2_e1_name_clean.wig | head -n 2
# Update for all chr

In [ ]:
# Extracting chromosomes
	sed -n 1,5127664p chm13_gemmap_l100_m2_e1_name_clean.wig \
	> chr1.wig
	sed -n 5127665,7917910p chm13_gemmap_l100_m2_e1_name_clean.wig \
	> chr2.wig
# Update for all chr

In [ ]:
# Replace chr name
	sed 's/chr1 CP068277.2 Homo sapiens isolate CHM13 chromosome 1/chr1/g' chr1.wig \
	> chr1_name_clean.wig 
	sed 's/chr2 CP068276.2 Homo sapiens isolate CHM13 chromosome 2/chr2/g' chr2.wig \
	> chr2_name_clean.wig 
#Update for all chromosomes
# Put all cleaned wig files together
	cat *_name_clean_wig > 	chm13_gemmap_l100_m2_e1_name_clean.wig

In [ ]:
# Rerun last past of script (L54-56)
	## Removing additional chromosome name information from fasta - to prevent errors when generating bed file
	## Dependencies
	export PATH=$PATH:/path/to/GEM-binaries-Linux-x86_64-core_i3-20130406-045632/bin
	export PATH=$PATH:/path/to/bedops/bin
	# Base ID for that stringency level
	MAPBASE=chm13_gemmap_l100_m2_e1
    
	wig2bed -m 16G < ${MAPBASE}_name_clean.wig > ${MAPBASE}.bed
	awk '$5>0.9' ${MAPBASE}.bed > ${MAPBASE}_uniq.bed

### Union validation script for  whole genome 

In [ ]:
# Creating bed file from reference CHM13
awk 'BEGIN {FS="\t"}; {print $1 FS "0" FS $2}' $MAIN/sv_hackathon/chm13.fa.fai > $MAIN/sv_hackathon/chm13.bed
head $MAIN/sv_hackathon/chm13.bed

In [ ]:
# Creating bed file from reference CHM13
awk '{FS="\t"};{print $1 FS $3}' $MAIN/sv_hackathon/chm13.bed > $MAIN/sv_hackathon/chm13_onlychr.genome
head $MAIN/sv_hackathon/chm13_onlychr.genome

In [ ]:
sbatch --mem=100g --cpus-per-task=4 \ 
	--mail-type=ALL --time=12:00:00 sort_union.sh \ 
	chm13_gemmap_l100_m2_e1_uniq.bed \ 
	chm13_gemmap_l250_m0_e0_uniq.bed

Done! 